<h1 style="background: orange; color: #5E7AFF; text-align: center; line-height: 200%; letter-spacing: 1px; font-weight: bold;">Möbius's Companion<br>
<span style="font-size: smaller; font-weight: normal;line-height: 150%;">Peter Luschny, April 2025</span></h1>

In [1]:
##############################################################################
## This is a SageMath Jupyter notebook, written in Python 3.13 with Sage 10.5.

from typing import Callable
from sage.all import ZZ, valuation, matrix, srange, cached_function, moebius, simplify, exp, log, divisors, factor, prime_divisors, prime_range, is_prime, sigma, sqrt, is_square

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Introduction</h1>

The basis of divisibility theory is given by the triangle A113704 and its inverse triangle, the Möbius triangle A363914, whose first column represents the Möbius function A008683. This theory considers divisibility without taking multiplicity of divisors into account.

The theory that considers the multiplicity of divisors is based on the divisibility triangle A382944 and its inverse triangle A382881, whose first column is the analog of the Möbius function and is represented by sequence A382883.

Just as the values of the Möbius function (0, 1, -1, not 0) lead to the decomposition (A013929, A030229, A030059, and A005117), the current sequence leads to the decomposition (A382943, A383016, A383017, and A383106).

The summatory function of A382883 is A382942, which is the counterpart of Mertens's function A002321. A382940(n) = A382883(n)*n is the counterpart of A055615, and A382941 is the counterpart of the exponential von Mangoldt function A014963.

Similar to the Möbius function, sequence A382883 can also be viewed as a transformation. Under this transformation, A000012 is mapped to A383104 and A000027 to A383124. The Dirichlet inverse of A382883 is A383210.

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Notation</h1>

<p style="color:brown;font-size:large">Note the following naming conventions:<br>

µ is the name of the Möbius sequence (A008683). <br>
ν is the name of the Möbius companion sequence (A382883).

µ is the Greek small letter mu and ν is the Greek small letter nu.<br>
We also use these Unicode characters as Python names and parts of Python names. 

ΣT is the sequence of row sums of the lower triangular matrix T. <br>
Σa is the sequence of partial sums of the sequence a. 

Given the arithmetic function $ \alpha $, we call the Dirichlet convolution $ \alpha * g $ the $ \alpha $-transform and write
$$
    \alpha T : (g, n) \mapsto \sum_{d \mid n} \alpha(d) \, g\!\left(\frac{n}{d}\right).
$$
Thus in particular µT is the Möbius transform and νT its companion transform.

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Valuation</h1>

<p style="color:brown;font-size:large">Valuation V(n, k) defined for n, k >= 0. For n, k >= 2 defined as the exponent of the highest power of k that divides n. Otherwise V(n, 0) = 0^n and V(n, 1) = 1. <i>Valuation</i> is also called the <i>p-adic order</i>. Outside the lower triangular matrix 0 <= k <= n all values are 0.</p>

In [2]:
@cached_function
def V(n: int, k: int) -> int:
    if k  > n: return 0
    if k == 0: return 0^n
    if k == 1: return 1 
    return valuation(n, k)

for n in range(13):
    print([n], [V(n, k) for k in srange(n + 1)])

[0] [1]
[1] [0, 1]
[2] [0, 1, 1]
[3] [0, 1, 0, 1]
[4] [0, 1, 2, 0, 1]
[5] [0, 1, 0, 0, 0, 1]
[6] [0, 1, 1, 1, 0, 0, 1]
[7] [0, 1, 0, 0, 0, 0, 0, 1]
[8] [0, 1, 3, 0, 1, 0, 0, 0, 1]
[9] [0, 1, 0, 2, 0, 0, 0, 0, 0, 1]
[10] [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]
[11] [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[12] [0, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">In the OEIS you will find it at this A-number:</p>

In [3]:
A382944 = V
print([A382944(n, k) for n in range(8) for k in srange(n+1)])

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">The row sums are the number of divisors of n, counted with their multiplicity.

In [4]:
def ΣV(n: int) -> int:
    return sum(V(n, k) for k in srange(n + 1))

print([ΣV(n) for n in range(1, 28)])

[1, 2, 2, 4, 2, 4, 2, 6, 4, 4, 2, 7, 2, 4, 4, 9, 2, 7, 2, 7, 4, 4, 2, 10, 4, 4, 6]


<p style="color:brown;font-size:large">In the OEIS this function is registered as:</p>

In [5]:
A169594 = ΣV

<p style="color:brown;font-size:large">The inverse triangle of V is defined recursively:

In [6]:
@cached_function
def invV(n:int, k:int) -> int:
    if n == k: return 1
    if k == 0: return 0^n
    return -sum(V(n, j)*invV(j, k) for j in range(1, n))

for n in range(13):
    print([n], [invV(n, k) for k in srange(n + 1)])

[0] [1]
[1] [0, 1]
[2] [0, -1, 1]
[3] [0, -1, 0, 1]
[4] [0, 1, -2, 0, 1]
[5] [0, -1, 0, 0, 0, 1]
[6] [0, 1, -1, -1, 0, 0, 1]
[7] [0, -1, 0, 0, 0, 0, 0, 1]
[8] [0, 1, -1, 0, -1, 0, 0, 0, 1]
[9] [0, 1, 0, -2, 0, 0, 0, 0, 0, 1]
[10] [0, 1, -1, 0, 0, -1, 0, 0, 0, 0, 1]
[11] [0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[12] [0, 0, 1, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">Let's check that this is indeed the inverse of V: 

In [7]:
M = matrix(ZZ, 13, V).inverse()
print(M)

[ 1  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0]
[ 0 -1  1  0  0  0  0  0  0  0  0  0  0]
[ 0 -1  0  1  0  0  0  0  0  0  0  0  0]
[ 0  1 -2  0  1  0  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  1  0  0  0  0  0  0  0]
[ 0  1 -1 -1  0  0  1  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  1  0  0  0  0  0]
[ 0  1 -1  0 -1  0  0  0  1  0  0  0  0]
[ 0  1  0 -2  0  0  0  0  0  1  0  0  0]
[ 0  1 -1  0  0 -1  0  0  0  0  1  0  0]
[ 0 -1  0  0  0  0  0  0  0  0  0  1  0]
[ 0  0  1  0 -1  0 -1  0  0  0  0  0  1]


<p style="color:brown;font-size:large">This is the A-number for the triangular array invV in the OEIS:

In [8]:
A382881 = invV
print([A382881(n, k) for n in range(8) for k in srange(n+1)])

[1, 0, 1, 0, -1, 1, 0, -1, 0, 1, 0, 1, -2, 0, 1, 0, -1, 0, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">The row sums give A000007 with offset 1, which is the identity function for Dirichlet multiplication.

In [9]:
def ΣVinv(n: int) -> int:
    return sum(invV(n, k) for k in srange(n + 1))

print([ΣVinv(n) for n in range(1, 20)])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Möbius's companion, called ν. </h1>

<p style="color:brown;font-size:large">Möbius's companion is defined as column 1 of the inverse of the valuation triangle A382944. This is analog to the definition of the Möbius function as column 1 of the inverse of the divisibility triangle A113704.

In [10]:
@cached_function
def _v(n: int) -> int:
    if n < 2: return n
    return -sum(V(n, j)*_v(j) for j in range(1, n))

print([_v(n) for n in srange(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">Slightly optimized by calling the valuation function directly this is:

In [11]:
@cached_function
def ν(n: int) -> int: 
    return (n if n < 2 else 
           -sum(1 if j == 1 
                else 0 if ν(j) == 0 
                else valuation(n, j)*ν(j) 
            for j in divisors(n)[:-1]))

print([ν(n) for n in srange(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">This is how you find it in the OEIS catalog:

In [12]:
A382883 = ν
print([A382883(n) for n in srange(1, 30)])
print([A382881(n, 1) for n in srange(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">The three fibers of ν.

In [13]:
def A382943List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == 0]

def A383017List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == -1]

def A383016List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == 1]

print(A382943List(60))
print(A383017List(60))
print(A383016List(60))

[12, 16, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 36, 37, 41, 42, 43, 47, 53, 59]
[1, 4, 6, 8, 9, 10, 14, 15, 21, 22, 25, 26, 27, 32, 33, 34, 35, 38, 39, 46, 49, 51, 55, 57, 58]


<p style="color:brown;font-size:large">... and, for convenience, also the case v(n) != 0:

In [14]:
def A383106List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) != 0]

print(A383106List(30))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 21, 22, 23, 25, 26, 27, 29]


<p style="color:brown;font-size:large">The Möbius function, called µ:

In [15]:
µ = moebius

<p style="color:brown;font-size:large">Its name in the OEIS is:

In [16]:
A008683 = µ
print([A008683(n) for n in range(1, 30)])

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large">Dirichlet inverse of positive integers:

In [17]:
def A382940(n:int) -> int: 
    return ν(n) * n

def A055615(n:int) -> int: 
    return µ(n) * n

print([A382940(n) for n in srange(1, 30)])
print([A055615(n) for n in srange(1, 30)])

[1, -2, -3, 4, -5, 6, -7, 8, 9, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 25, 26, 27, 0, -29]
[1, -2, -3, 0, -5, 6, -7, 0, 0, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 0, 26, 0, 0, -29]


<p style="color:brown;font-size:large">Family similarities and differences between the Möbius function and his companion are captured in A383105, A383103, and A383018.

In [18]:
print([n for n in range(1, 362) if µ(n) != ν(n)])             # A383105
print([n for n in range(1, 362) if µ(n) == 0 and ν(n) == 1])  # A383103
print([n for n in range(1, 362) if µ(n) == 0 and ν(n) == -1]) # A383018

[4, 8, 9, 25, 27, 32, 36, 49, 64, 100, 121, 125, 128, 169, 196, 216, 225, 243, 289, 343, 361]
[4, 8, 9, 25, 27, 32, 49, 121, 125, 128, 169, 243, 289, 343, 361]
[36, 64, 100, 196, 216, 225]


<p style="color:brown;font-size:large">Note that ν(n) = 0 implies µ(n) = 0. In other words, if ν(n) = 0 then n is divisible by the square of a prime.<br><br>
ν(n) = 0  <=> n is A059404 or n is A217261.<br>
A059404 Numbers with different exponents in their prime factorizations.<br>
A217261 Numbers of the form i^j^k, for i, j, k > 1.

In [19]:
# This is a test that v(n) = 0 implies µ(n) = 0.

A = [n for n in range(1, 4362) if ν(n) == 0]
set([µ(n) for n in A])

{0}

In [20]:
# This is a test that v vanishes on A059404.

def isA059404(n) -> bool: 
    return 1 < len(set(valuation(n, p) for p in prime_divisors(n)))
A = [n for n in range(1, 190) if isA059404(n)]

set([ν(n) for n in A])

{0}

In [21]:
# Data from A217261. The implementation of v is not efficient enough to compute large terms.

A = [16, 81, 256, 512, 625, 1296, 2401, 4096]
B = [16, 81, 256, 512, 625, 1296, 2401, 4096, 6561, 10000, 14641, 19683, 20736, 28561, 38416, 50625, 65536, 83521, 104976, 130321, 160000, 194481, 234256, 262144, 279841, 331776, 390625, 456976, 531441, 614656, 707281, 810000, 923521, 1048576, 1185921]

set([ν(n) for n in A])
# for n in B: print(n, factor(n), µ(n), is_square(n))

{0}

<p style="color:brown;font-size:large">Note that ν(n) = -1 implies n is not in A053810. In other words, if ν(n) = -1 then n does not have the form p^e where both p and e are prime numbers; thus in particular n is not a prime number.<br>

In [22]:
# Test that v(n) == -1 implies n is not in A053810. 

def isA053810(n: int) -> bool:
    p = prime_divisors(n)
    return len(p) == 1 and is_prime(valuation(n, p[0]))

A = [n for n in range(1, 2000) if ν(n) == -1]
set([isA053810(n) for n in A])

{False}

<p style="color:brown;font-size:large">Illustrating the different indicator functions:

In [23]:
for n in range(1, 40):
    vn = ν(n); m = moebius(n); 
    pp = int(isA053810(n)); f = factor(n)
    print(f"{n:>3} | {vn:>2} | {m:>2} | {pp:>2} | {str(f):<12}") 

  1 |  1 |  1 |  0 | 1           
  2 | -1 | -1 |  0 | 2           
  3 | -1 | -1 |  0 | 3           
  4 |  1 |  0 |  1 | 2^2         
  5 | -1 | -1 |  0 | 5           
  6 |  1 |  1 |  0 | 2 * 3       
  7 | -1 | -1 |  0 | 7           
  8 |  1 |  0 |  1 | 2^3         
  9 |  1 |  0 |  1 | 3^2         
 10 |  1 |  1 |  0 | 2 * 5       
 11 | -1 | -1 |  0 | 11          
 12 |  0 |  0 |  0 | 2^2 * 3     
 13 | -1 | -1 |  0 | 13          
 14 |  1 |  1 |  0 | 2 * 7       
 15 |  1 |  1 |  0 | 3 * 5       
 16 |  0 |  0 |  0 | 2^4         
 17 | -1 | -1 |  0 | 17          
 18 |  0 |  0 |  0 | 2 * 3^2     
 19 | -1 | -1 |  0 | 19          
 20 |  0 |  0 |  0 | 2^2 * 5     
 21 |  1 |  1 |  0 | 3 * 7       
 22 |  1 |  1 |  0 | 2 * 11      
 23 | -1 | -1 |  0 | 23          
 24 |  0 |  0 |  0 | 2^3 * 3     
 25 |  1 |  0 |  1 | 5^2         
 26 |  1 |  1 |  0 | 2 * 13      
 27 |  1 |  0 |  1 | 3^3         
 28 |  0 |  0 |  0 | 2^2 * 7     
 29 | -1 | -1 |  0 | 29          
 30 | -1 | -1 

In [24]:
# And there are more things to study ...
# A162143 or A074853 ?

<p style="color:brown;font-size:large">The summatory function of ν ...

In [25]:
def Σν(n: int) -> int:
    return sum(ν(k) for k in range(n+1))

print([Σν(n) for n in range(1, 30)])

[1, 0, -1, 0, -1, 0, -1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 3, 4, 4, 3]


<p style="color:brown;font-size:large"> ... and its ID in the OEIS:

In [26]:
A382942 = Σν
print([A382942(n) for n in range(1, 30)])

[1, 0, -1, 0, -1, 0, -1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 3, 4, 4, 3]


<p style="color:brown;font-size:large">The von Mangoldt brothers join the Moebius party:

In [27]:
def νMangoldt(n: int) -> int:
    return simplify(exp(sum(ν(d)*log(n//d) for d in divisors(n))))

def µMangoldt(n: int) -> int:
    return simplify(exp(sum(µ(d)*log(n//d) for d in divisors(n))))

In [28]:
print([νMangoldt(n) for n in srange(1, 30)])    # A014963
print([µMangoldt(n) for n in srange(1, 30)])    # A382941

[1, 2, 3, 2, 5, 1, 7, 4, 3, 1, 11, 3, 13, 1, 1, 16, 17, 2, 19, 5, 1, 1, 23, 18, 5, 1, 9, 7, 29]
[1, 2, 3, 2, 5, 1, 7, 2, 3, 1, 11, 1, 13, 1, 1, 2, 17, 1, 19, 1, 1, 1, 23, 1, 5, 1, 3, 1, 29]


<p style="color:brown;font-size:large">On OEIS use the names:

In [29]:
A382941 = νMangoldt
A014963 = µMangoldt

<p style="color:brown;font-size:large">Summatory functions:

In [30]:
from itertools import accumulate

def ΣνMangoldt(len: int) -> list[int]:
    return list(accumulate([νMangoldt(n) for n in srange(1, len)]))

def ΣµMangoldt(len: int) -> list[int]:
    return list(accumulate([µMangoldt(n) for n in srange(1, len)]))

In [31]:
print(ΣνMangoldt(22))
print(ΣµMangoldt(22))

[1, 3, 6, 8, 13, 14, 21, 25, 28, 29, 40, 43, 56, 57, 58, 74, 91, 93, 112, 117, 118]
[1, 3, 6, 8, 13, 14, 21, 23, 26, 27, 38, 39, 52, 53, 54, 56, 73, 74, 93, 94, 95]


<p style="color:brown;font-size:large">On OEIS you see this:

In [32]:
missing = ΣνMangoldt
A072107 = ΣµMangoldt

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Transforms à la Dirichlet</h1>

In [33]:
def vT(b, n) -> int:
    return sum(ν(n//d)*b(d) for d in divisors(n))

def µT(b, n) -> int:
    return sum(µ(n//d)*b(d) for d in divisors(n))

<p style="color:brown;font-size:large">Three classical examples for the transforms:

* b: x -> 1 <br>
* b: x -> x <br>
* b: x -> 1 if x = 1 else 0

In [34]:
b = lambda x: 1

print([vT(b, n) for n in srange(1, 30)])
print([µT(b, n) for n in srange(1, 30)])

[1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<p style="color:brown;font-size:large">On the OEIS you see this: A383104, similar to A366988; apparently with the same indicator function for n > 1.

In [35]:
b = lambda x: x

print([vT(b, n) for n in srange(1, 30)])
print([µT(b, n) for n in srange(1, 30)])

[1, 1, 2, 3, 4, 2, 6, 7, 7, 4, 10, 7, 12, 6, 8, 14, 16, 8, 18, 13, 12, 10, 22, 17, 21, 12, 22, 19, 28]
[1, 1, 2, 2, 4, 2, 6, 4, 6, 4, 10, 4, 12, 6, 8, 8, 16, 6, 18, 8, 12, 10, 22, 8, 20, 12, 18, 12, 28]


<p style="color:brown;font-size:large">This is A383124 in the case ν and Euler's totient function A000010 in the case µ.

In [36]:
b = lambda x: 1 if x == 1 else 0

print([vT(b, n) for n in srange(1, 30)])
print([µT(b, n) for n in srange(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large">The first sequence is missing in OEIS and the second is µ A008683.

<p style="color:brown;font-size:large">Implemented as sequence-to-sequence transformations:

In [37]:
def νT(b: Callable[[int], int]) -> Callable[[int], int]:
    def νb(n: int) -> int:
        return sum(ν(n//d)*b(d) for d in divisors(n))
    return νb

In [38]:
ν1 = νT(lambda _: 1); 
νX = νT(lambda x: x); 
νI = νT(lambda x: 1 if x == 1 else 0)

print([ν1(n) for n in range(1, 30)])
print([νX(n) for n in range(1, 30)])
print([νI(n) for n in range(1, 30)])

[1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0]
[1, 1, 2, 3, 4, 2, 6, 7, 7, 4, 10, 7, 12, 6, 8, 14, 16, 8, 18, 13, 12, 10, 22, 17, 21, 12, 22, 19, 28]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


In [39]:
def µT(b: Callable[[int], int]) -> Callable[[int], int]:
    def µb(n: int) -> int:
        return sum(µ(n//d)*b(d) for d in divisors(n))
    return µb

In [40]:
µ1 = µT(lambda _: 1); 
µX = µT(lambda x: x); 
µI = µT(lambda x: 1 if x == 1 else 0)

print([µ1(n) for n in range(1, 30)])
print([µX(n) for n in range(1, 30)])
print([µI(n) for n in range(1, 30)])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 2, 2, 4, 2, 6, 4, 6, 4, 10, 4, 12, 6, 8, 8, 16, 6, 18, 8, 12, 10, 22, 8, 20, 12, 18, 12, 28]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large;">Let's check a special case:</p>

In [41]:
νµ = νT(µ)
µν = µT(ν)

print([νµ(n) for n in range(1, 30)]) 
print([µν(n) for n in range(1, 30)]) 

[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]
[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]


<p style="color:brown;font-size:large;">This is not surprising as the Dirichlet convolution is commutative but we are pleased that it also applies in our case (since we found on two different paths).

$$
     \nu * \mu = \mu * \nu  
$$

$$
νT(µ) = µT(ν)
$$

$$
    \sum_{d \mid n} \nu(d) \, \mu\!\left(\frac{n}{d}\right) =
    \sum_{d \mid n} \mu(d) \, \nu\!\left(\frac{n}{d}\right)
$$

<p style="color:brown;font-size:large">In OEIS it has the form A383123:

In [42]:
@cached_function
def A382883(n: int) -> int: 
    return n if n < 2 else -sum((1 if j == 1 else valuation(n, j))*A382883(j) 
                            for j in divisors(n)[:-1])

def vT(b: Callable[[int], int]) -> Callable[[int], int]:
    @cached_function
    def vb(n: int) -> int:
        return sum(A382883(n//d)*b(d) for d in divisors(n))
    return vb

A383123 = vT(moebius)
print([A383123(n) for n in range(1, 30)])

[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]


<p style="color:brown;font-size:large">We could iterate:

In [43]:
A = vT(vT(moebius))
print([A(n) for n in range(1, 30)])

B = µT(µT(moebius))
print([B(n) for n in range(1, 30)])

[1, -3, -3, 5, -3, 9, -3, -3, 5, 9, -3, -13, -3, 9, 9, -1, -3, -13, -3, -13, 9, 9, -3, 7, 5, 9, -3, -13, -3]
[1, -3, -3, 3, -3, 9, -3, -1, 3, 9, -3, -9, -3, 9, 9, 0, -3, -9, -3, -9, 9, 9, -3, 3, 3, 9, -1, -9, -3]


In [44]:
A = vT(µT(moebius))
B = µT(vT(moebius))

print([A(n) for n in range(1, 30)])
print([B(n) for n in range(1, 30)])

[1, -3, -3, 4, -3, 9, -3, -2, 4, 9, -3, -11, -3, 9, 9, -1, -3, -11, -3, -11, 9, 9, -3, 5, 4, 9, -2, -11, -3]
[1, -3, -3, 4, -3, 9, -3, -2, 4, 9, -3, -11, -3, 9, 9, -1, -3, -11, -3, -11, 9, 9, -3, 5, 4, 9, -2, -11, -3]


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
The Dirichlet Inverse</h1>

In [45]:
def iT(b: Callable[[int], int]) -> Callable[[int], int]:
    """We assume b(1) = 1!"""
    @cached_function
    def g(n: int) -> int:
        if n == 1:
            return 1
        s = sum(g(n//d)*b(d) for d in divisors(n)[1:])
        return -s
    return g

<p style="color:brown;font-size:large">The Dirichlet inverse of ν is iν and the Dirichlet inverse of µ is iµ.

In [46]:
iν = iT(ν) 
iµ = iT(µ) 

# ... and back again
iiν = iT(iν)
iiµ = iT(iµ)

In [47]:
print([iν(n) for n in range(1, 30)])
print([iµ(n) for n in range(1, 30)])

[1, 1, 1, 0, 1, 1, 1, -2, 0, 1, 1, -1, 1, 1, 1, -3, 1, -1, 1, -1, 1, 1, 1, -5, 0, 1, -2, -1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [48]:
print([ν(n) for n in range(1, 30)])
print([iiν(n) for n in range(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


In [49]:
print([µ(n) for n in range(1, 30)])
print([iiµ(n) for n in range(1, 30)])

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large"> A383210 / A000012 / A382883 /  A008683

In [50]:
i1 = iT(lambda _: 1) 
ix = iT(lambda x: x) 
iI = iT(lambda x: 1 if x == 1 else 0)

print([i1(n) for n in range(1, 30)])
print([ix(n) for n in range(1, 30)])
print([iI(n) for n in range(1, 30)])

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]
[1, -2, -3, 0, -5, 6, -7, 0, 0, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 0, 26, 0, 0, -29]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<p style="color:brown;font-size:large"> A008683 / A055615 / A000007

In [51]:
i1 = iT(νT(lambda _: 1))
ix = iT(νT(lambda x: x))
iI = iT(νT(lambda x: 1 if x == 1 else 0))

print([i1(n) for n in range(1, 30)])
print([ix(n) for n in range(1, 30)])
print([iI(n) for n in range(1, 30)])

[1, 0, 0, -1, 0, 0, 0, -2, -1, 0, 0, -1, 0, 0, 0, -1, 0, -1, 0, -1, 0, 0, 0, -2, -1, 0, -2, -1, 0]
[1, -1, -2, -2, -4, 2, -6, -2, -3, 4, -10, 3, -12, 6, 8, 1, -16, 2, -18, 7, 12, 10, -22, 3, -5, 12, -2, 11, -28]
[1, 1, 1, 0, 1, 1, 1, -2, 0, 1, 1, -1, 1, 1, 1, -3, 1, -1, 1, -1, 1, 1, 1, -5, 0, 1, -2, -1, 1]


<p style="color:brown;font-size:large"> ? / ? / A383211

In [52]:
i1 = iT(µT(lambda _: 1))
ix = iT(µT(lambda x: x))
iI = iT(µT(lambda x: 1 if x == 1 else 0))

print([i1(n) for n in range(1, 30)])
print([ix(n) for n in range(1, 30)])
print([iI(n) for n in range(1, 30)])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, -1, -2, -1, -4, 2, -6, -1, -2, 4, -10, 2, -12, 6, 8, -1, -16, 2, -18, 4, 12, 10, -22, 2, -4, 12, -2, 6, -28]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p style="color:brown;font-size:large">A000007 / A023900 / A000012

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Some uses in classifying integers</h1>

<p style="color:brown;font-size:large">Examples:  

Recall A382883 = ν.

ν(99) = 0 because 99 is a number with different exponents in the prime factorization (A059404).

ν(2197) = 1 because 2197 is a number of the form p^e where both p and e are prime numbers (A053810).

ν(127) = -1 because 127 is a prime number.

<p style="color:brown;font-size:large">A059404

In [53]:
@cached_function
def isA059404(n: int) -> bool: 
    return 1 < len(set(valuation(n, p) for p in prime_divisors(n)))

F = [n for n in range(1, 2000) if isA059404(n)]
set([ν(n) for n in F])

{0}

<p style="color:brown;font-size:large">A053810

In [54]:
def isA053810(n: int) -> bool:
    p = prime_divisors(n)
    return len(p) == 1 and is_prime(valuation(n, p[0]))

F = [n for n in srange(1, 2000) if isA053810(n)]
set([ν(n) for n in F])

{1}

<p style="color:brown;font-size:large">A000040

In [55]:
F = [p for p in prime_range(50)]

print([µ(p) for p in F])
print([ν(p) for p in F])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


<p style="color:brown;font-size:large">A001248

In [56]:
F = [p^2 for p in prime_range(50)]

print([µ(p) for p in F])
print([ν(p) for p in F])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p style="color:brown;font-size:large">Similar but different from A259183


In [57]:
F = [n for n in srange(1, 150) if µ(n) != ν(n)]
print(F)

[4, 8, 9, 25, 27, 32, 36, 49, 64, 100, 121, 125, 128]


<p style="color:brown;font-size:large">Similar but different from A323350

In [58]:
def bigomega(n: int) -> int: 
    return sloane.A001222(n)
F = [bigomega(n) for n in srange(1, 100)]

print([µ(n) for n in F])
print([ν(n) for n in F])
print([n for n in srange(1, 260) if µ(bigomega(n)) != ν(bigomega(n))])

[0, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 0, 1, -1, 1, -1, -1, -1, 1, 0, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 0, 1, -1, -1, 0, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 0, -1, 0, -1, -1, 1, 0, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 0, -1, 1, 0, -1, -1, -1, 0, 1, 0, -1, -1, -1, -1, -1, 1, 1, -1, -1]
[0, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1, -1]
[16, 24, 36, 40, 54, 56, 60, 81, 84, 88, 90, 100, 104, 126, 132, 135, 136, 140, 150, 152, 156, 184, 189, 196, 198, 204, 210, 220, 225, 228, 232, 234, 248, 250, 256]


<p style="color:brown;font-size:large">A120497 ? Positive integers whose number of divisors is a perfect power ?

In [59]:
def tau(n: int) -> int: 
    return sigma(n, 0)
print([n for n in srange(1, 50) if µ(tau(n)) != ν(tau(n))])

[6, 8, 10, 14, 15, 21, 22, 24, 26, 27, 30, 33, 34, 35, 36, 38, 39, 40, 42, 46]


<p style="color:brown;font-size:large">Subsequence of A065496 ? 

In [60]:
def s1(n: int) -> int: 
    return sigma(n, 1)
print([n for n in srange(1, 150) if µ(s1(n)) != ν(s1(n))])

[3, 7, 21, 22, 31, 81, 93, 102, 110, 127, 142]


<p style="color:brown;font-size:large">Clarify their relation to the numbers A059404 with different exponents in their prime factorizations.

In [61]:
def isA059404(n: int) -> bool: 
    return 1 < len(set(valuation(n, p) for p in prime_divisors(n)))

B = [n for n in range(1, 4362) if isA059404(n)]
print(B)

[12, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56, 60, 63, 68, 72, 75, 76, 80, 84, 88, 90, 92, 96, 98, 99, 104, 108, 112, 116, 117, 120, 124, 126, 132, 135, 136, 140, 144, 147, 148, 150, 152, 153, 156, 160, 162, 164, 168, 171, 172, 175, 176, 180, 184, 188, 189, 192, 198, 200, 204, 207, 208, 212, 220, 224, 228, 232, 234, 236, 240, 242, 244, 245, 248, 250, 252, 260, 261, 264, 268, 270, 272, 275, 276, 279, 280, 284, 288, 292, 294, 296, 297, 300, 304, 306, 308, 312, 315, 316, 320, 324, 325, 328, 332, 333, 336, 338, 340, 342, 344, 348, 350, 351, 352, 356, 360, 363, 364, 368, 369, 372, 375, 376, 378, 380, 384, 387, 388, 392, 396, 400, 404, 405, 408, 412, 414, 416, 420, 423, 424, 425, 428, 432, 436, 440, 444, 448, 450, 452, 456, 459, 460, 464, 468, 472, 475, 476, 477, 480, 486, 488, 490, 492, 495, 496, 500, 504, 507, 508, 513, 516, 520, 522, 524, 525, 528, 531, 532, 536, 539, 540, 544, 548, 549, 550, 552, 556, 558, 560, 564, 567, 568, 572, 575, 576, 578, 580, 584, 585, 588, 592, 594, 596, 6

<p style="color:brown;font-size:large">The numbers A217261 of the form i^j^k, for i, j, k > 1 ? How are they related to A372405, the exponentially powerful numbers whose prime factorization exponents are all powerful numbers > 1.

In [62]:
MAX = 1185922
#MAX = 100000000

A217261List = sorted(set([x for x in [i^(j^2) 
              for j in range(2, 120) 
              for i in range(2, 120)] 
              if x < MAX]))

print(A217261List)

[16, 81, 256, 512, 625, 1296, 2401, 4096, 6561, 10000, 14641, 19683, 20736, 28561, 38416, 50625, 65536, 83521, 104976, 130321, 160000, 194481, 234256, 262144, 279841, 331776, 390625, 456976, 531441, 614656, 707281, 810000, 923521, 1048576, 1185921]


In [63]:
i=1
for n in A217261List:
    print([i], n, µ(n), sqrt(n))
    i += 1

[1] 16 0 4
[2] 81 0 9
[3] 256 0 16
[4] 512 0 16*sqrt(2)
[5] 625 0 25
[6] 1296 0 36
[7] 2401 0 49
[8] 4096 0 64
[9] 6561 0 81
[10] 10000 0 100
[11] 14641 0 121
[12] 19683 0 81*sqrt(3)
[13] 20736 0 144
[14] 28561 0 169
[15] 38416 0 196
[16] 50625 0 225
[17] 65536 0 256
[18] 83521 0 289
[19] 104976 0 324
[20] 130321 0 361
[21] 160000 0 400
[22] 194481 0 441
[23] 234256 0 484
[24] 262144 0 512
[25] 279841 0 529
[26] 331776 0 576
[27] 390625 0 625
[28] 456976 0 676
[29] 531441 0 729
[30] 614656 0 784
[31] 707281 0 841
[32] 810000 0 900
[33] 923521 0 961
[34] 1048576 0 1024
[35] 1185921 0 1089


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Appendix</h1>

In [64]:
%%time

[ν(n) for n in range(1, 20000)];

CPU times: user 854 ms, sys: 7.93 ms, total: 862 ms
Wall time: 893 ms


In [65]:
%%time

[µ(n) for n in range(1, 20000)];

CPU times: user 129 ms, sys: 9.43 ms, total: 138 ms
Wall time: 141 ms


In [66]:
c0 = c1 = c2 = 0
for n in range(1, 20000):
    vn = ν(n)
    if vn == 0: c0 +=1
    elif vn == 1: c1 +=1
    else: c2 += 1    
print(c0, c1, c2)

7723 6155 6121


In [67]:
c0 = c1 = c2 = 0
for n in range(1, 20000):
    µn = µ(n)
    if µn == 0: c0 +=1
    elif µn == 1: c1 +=1
    else: c2 += 1    
print(c0, c1, c2)

7839 6093 6067
